# Pruebas Unitarias e Integrales_Modelo Nivel de Madurez MOTI

**Autor**: Johan David Ramirez Hernandez
           Jonathan Alexander Torres Benitez
           Paula Andrea Escobar López
           Jorge Ivan Gomez
 

### Paquetes

Instalación de paquetes que consideramos relevantes para realizar el ejercicio.

In [1]:
# !pip install boto3
# !pip install shapely
# !pip install hdbscan
# !pip install pycaret
# !pip install scikit-learn
# !pip install pandas
# !pip install pivottablejs

Importar librerías necesarias para el desarrollo del modelo

In [2]:
# Operating System
import os
import boto3

# Packages for strings
import re
from unidecode import unidecode

# Packages for data analysis
import numpy as np
import pandas as pd
import shapely
import sklearn
import unittest
import hdbscan

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

# Pacakges for Data Science
from sklearn import preprocessing
import hdbscan
import pycaret
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.cluster import OPTICS
from pycaret import clustering
from pivottablejs import pivot_ui

# Packages for visualizations
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
import random

warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

# Define la fuente alternativa
font_name = "DejaVu Sans"  # Nombre de la fuente alternativa

# Configura la fuente alternativa para Matplotlib
plt.rc('font', family=font_name)

# Funciones 

Función para el reemplzado de outliers (LS - LI)

# Importar datos

Obtener el datset desde el bucket s3

In [3]:
s3 = boto3.client('s3', region_name='us-east-1')

In [4]:
bucket_name = 'evolucion-modelo-operativo'
key_1 = 'Modelo_analiticio_Medicion_Madurez/df_medicion_beneficio.csv'

In [5]:
response = s3.get_object(Bucket=bucket_name, Key=key_1)
df = pd.read_csv(response['Body'])

# Generación de datos de prueba

In [6]:
df_numeric = df.select_dtypes(include=np.number)

In [7]:
df_numeric.drop(columns=['Unnamed: 0'], inplace=True)

In [8]:
def randomizador(df):
    get_random_value = lambda min_val, max_val: random.uniform(min_val, max_val)
    get_random_tricky = lambda min_val, max_val: random.uniform(0.7, 1)
    

    
    df_prueba = df.apply(lambda col: col.apply(lambda x: get_random_tricky(col.min(), col.max()) if col.name=='%Disponibilidad' else get_random_value(col.min(), col.max())))
    return df_prueba

In [9]:
df_datos_prueba = randomizador(df_numeric)

In [10]:
df_datos_prueba.head()

,"¿Cómo valoras tu satisfacción al usar las capacidades del MO TI en tu día a día; las consideras sencillas, te brindan autonomía y te generan valor?",%Disponibilidad,Min Afectación,Baja,Media,Critica,mttr_hr,Cumple ANS,LT,PT,...,Sentimiento de temor o ansiedad por los cambios que traerá esta transformación,No existe cultura de medición,El cambio no tiene la suficiente credibilidad al momento de generar resultados,El modelo de toma de decisiones para este cambio no es claro,Existen barreras de comunicación en la organización que afectan este cambio,Los objetivos que busca este cambio no están alineados con los objetivos de la organización,Ubicacion Infra,Beneficio Cloud,Medición Beneficio Cloud,Nivel Madurez
0,9.244986,0.717818,1116.118129,193.614761,0.563451,3.342518,12401.711779,359.869703,140.775018,6.859220,...,1.812214,0.570927,1.146723,1.139569,0.014494,0.740456,1.192995,0.324761,2.048303,3.025789
1,6.090016,0.738122,261.214229,344.037607,0.474665,3.064589,13333.324765,57.974113,294.895832,55.574296,...,0.791373,1.004749,2.737796,2.862608,0.509414,1.493791,0.685136,1.929133,2.621908,2.523747
2,7.464064,0.703363,463.566241,47.367126,0.357291,2.498853,9659.948314,199.321734,318.948343,87.690862,...,0.524555,0.844408,1.887977,0.950075,1.300062,0.230883,2.713405,2.603150,2.798101,2.138005
3,6.088141,0.933732,591.821043,54.643088,4.104018,0.948297,4259.500300,25.070542,227.307477,265.416157,...,0.675470,0.307902,1.091377,3.412343,3.155704,1.023094,3.224375,2.707757,2.968872,3.073587
4,6.251335,0.723693,155.773105,182.048367,0.599990,0.342618,2673.867966,71.583213,31.027444,312.840594,...,0.393981,0.951019,0.579118,1.004391,2.115086,2.100588,0.863825,2.534946,2.675165,2.456030


In [11]:
df_datos_prueba['cod app'] = df['cod app']

In [12]:
csv_filename = 'df_datos_prueba.csv'
folder_name = 'Modelo_analiticio_Medicion_Madurez'
df.to_csv(csv_filename)

s3_key = f'{folder_name}/{csv_filename}'

try:
    s3.upload_file(csv_filename, bucket_name, s3_key)
    print(f"{csv_filename} cargado exitosamente en la carpeta {folder_name} en S3.")
except NoCredentialsError:
    print("No se encontraron credenciales de AWS o se han configurado incorrectamente.")

df_datos_prueba.csv cargado exitosamente en la carpeta Modelo_analiticio_Medicion_Madurez en S3.


# Carga dataset para ML

In [13]:
key_2 = 'Modelo_analiticio_Medicion_Madurez/df_datos_prueba.csv'
response_2 = s3.get_object(Bucket=bucket_name, Key=key_2)
df_medicion_beneficio = pd.read_csv(response_2['Body'])

In [14]:
df = df_medicion_beneficio.copy()

In [15]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'cod app',
       '¿Cómo valoras tu satisfacción al usar las capacidades del MO TI en tu día a día; las consideras sencillas, te brindan autonomía y te generan valor?',
       '%Disponibilidad', 'Min Afectación', 'Baja', 'Media', 'Critica',
       'mttr_hr', 'Cumple ANS', 'LT', 'PT', 'Incidentes Onpremise',
       'Incidentes Nube', 'Cambios Onpremise', 'Cambios Nube',
       'Continuidad Onpremise', 'Continuidad Nube', 'Configuración',
       'Habilitar y Mantener Soluciones', 'AMI y Parchado', 'Operaciones TI',
       'Backup & Archiving', 'Optimizar los costos nube',
       'Administrar etiquetas',
       'Administrar estructura de organización y cuentas nube',
       'Nombre del Servicio', 'Nombre de EVC',
       'Falta de tiempo de los líderes',
       'Poco entrenamiento en los temas que implica este cambio',
       'El momento que vive la organización no es pertinente para realizar el cambio',
       'Sentimiento de temor o ansiedad por los c

In [16]:
df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'Ciclo de medición'], inplace=True)

# Pruebas Unitarias e Integrales

In [17]:
import pandas as pd
import hdbscan
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import pycaret.clustering as clustering
import pytest


def load_and_preprocess_data(df):
    data_scale = df.copy()
    data_scale.iloc[:, 1:] = preprocessing.MinMaxScaler().fit_transform(data_scale.iloc[:, 1:])
    
    n_components = 3
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(data_scale.iloc[:, 1:])
    X_pca = pd.DataFrame(X_pca)
    
    
    data_scale = pd.concat([data_scale[['cod app']], X_pca], axis=1)
    return data_scale


In [18]:
load_and_preprocess_data(df)

,cod app,0,1,2
0,NU0364001,-0.151368,-0.131784,0.167600
1,AW0450001,-0.425596,0.419982,-0.487493
2,AW1152001,-0.604280,-0.960614,-0.178071
3,NU0109001,0.238822,0.467713,-1.077319
4,AW1182001,-0.525706,0.573947,-0.295429
...,...,...,...,...
186,AW1203001,-0.596942,-1.053194,-0.104257
187,AW1197001,-0.476636,0.446195,-0.268370
188,AW0823001,-0.600660,-1.038863,-0.059322
189,AW1112001,-0.612370,-1.042208,-0.056639


In [19]:
def train_hdbscan_model(data_scale):
    s = clustering.setup(
        data=data_scale,
        index='cod app',
        preprocess=False,
        imputation_type=None,
        remove_multicollinearity=False,
        normalize=False,
        pca=False,
        session_id=123
    )
    
    hdbscan_model = clustering.create_model(
        hdbscan.HDBSCAN(),
        min_cluster_size=15,
        min_samples=2
    )
    
    return s, hdbscan_model  



In [20]:
import pandas as pd
import hdbscan
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import pycaret.clustering as clustering
import unittest


class TestClusteringModel(unittest.TestCase):

    def test_data_preprocessing(self):
        processed_data = load_and_preprocess_data(df)
        self.assertEqual(processed_data.shape[0], df.shape[0])

    def test_train_and_evaluate_model(self):
        processed_data = load_and_preprocess_data(df)
        s, hdbscan_model = train_hdbscan_model(processed_data)


if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


.

,Description,Value
0,Session id,123
1,Original data shape,"(191, 3)"
2,Transformed data shape,"(191, 3)"
3,Numeric features,3


,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.5340,141.6597,1.3378,0,0,0


.
----------------------------------------------------------------------
Ran 2 tests in 1.118s

OK


## Encriptación de dataset de Clusters

In [21]:
s3 = boto3.client('s3', region_name='us-east-1')

In [22]:
bucket_name = 'evolucion-modelo-operativo'
key_1 = 'Modelo_analiticio_Medicion_Madurez/cluster_madurez_MOTI.csv'

In [23]:
response = s3.get_object(Bucket=bucket_name, Key=key_1)
df_clusters = pd.read_csv(response['Body'])

In [24]:
df_clusters

,Unnamed: 0,cod app,"¿Cómo valoras tu satisfacción al usar las capacidades del MO TI en tu día a día; las consideras sencillas, te brindan autonomía y te generan valor?",%Disponibilidad,Min Afectación,Baja,Media,Critica,mttr_hr,Cumple ANS,...,No existe cultura de medición,El cambio no tiene la suficiente credibilidad al momento de generar resultados,El modelo de toma de decisiones para este cambio no es claro,Existen barreras de comunicación en la organización que afectan este cambio,Los objetivos que busca este cambio no están alineados con los objetivos de la organización,Ubicacion Infra,Beneficio Cloud,Medición Beneficio Cloud,Nivel Madurez,Cluster
0,0,NU0364001,8.0,0.995152,167.166667,0.333333,0.000000,0.333333,67.221898,0.666667,...,0,0,0,0,0,3,2,3.490919,2.8,3
1,1,AW0450001,8.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000010,0.000000,...,0,0,0,0,0,3,3,3.222131,3.2,4
2,2,AW1152001,8.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000010,0.000000,...,0,0,0,0,0,3,2,3.679435,2.8,1
3,3,NU0109001,6.0,0.000000,0.000000,45.000000,3.833333,0.166667,2297.309398,49.000000,...,1,1,0,1,0,3,3,2.489784,3.2,-1
4,4,AW1182001,8.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000010,0.000000,...,0,0,0,0,0,3,1,2.892218,2.4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,186,AW1203001,8.0,0.997928,92.500000,0.166667,0.000000,0.000000,9.023148,0.166667,...,0,0,0,0,0,2,2,3.108554,2.8,1
187,187,AW1197001,8.0,1.000000,0.000000,0.500000,0.000000,0.000000,4.978750,0.500000,...,0,0,0,0,0,4,2,2.892218,2.8,4
188,188,AW0823001,8.0,0.996094,161.833333,3.000000,0.333333,0.000000,274.104630,3.333333,...,0,0,0,0,0,3,2,3.679382,2.8,1
189,189,AW1112001,8.0,1.000000,0.000000,0.333333,0.000000,0.166667,21.267083,0.500000,...,0,0,0,0,0,3,2,3.679435,2.8,1


In [25]:
df_clusters.drop(columns=['Unnamed: 0'], inplace=True)

In [26]:
import hashlib

# Define una función para enmascarar los valores
def enmascarar_valor(valor):
    sha256 = hashlib.sha256()
    sha256.update(str(valor).encode())
    return sha256.hexdigest()

# Aplica la función de enmascaramiento a la columna 'cod app'
df_clusters['cod app'] = df_clusters['cod app'].apply(enmascarar_valor)




In [27]:
print("\nDataFrame Encriptado:")
df_clusters


DataFrame Encriptado:


,cod app,"¿Cómo valoras tu satisfacción al usar las capacidades del MO TI en tu día a día; las consideras sencillas, te brindan autonomía y te generan valor?",%Disponibilidad,Min Afectación,Baja,Media,Critica,mttr_hr,Cumple ANS,LT,...,No existe cultura de medición,El cambio no tiene la suficiente credibilidad al momento de generar resultados,El modelo de toma de decisiones para este cambio no es claro,Existen barreras de comunicación en la organización que afectan este cambio,Los objetivos que busca este cambio no están alineados con los objetivos de la organización,Ubicacion Infra,Beneficio Cloud,Medición Beneficio Cloud,Nivel Madurez,Cluster
0,eea15b72e72c0034b32fb3154a07a273e451a437dc8209...,8.0,0.995152,167.166667,0.333333,0.000000,0.333333,67.221898,0.666667,64.646553,...,0,0,0,0,0,3,2,3.490919,2.8,3
1,423b3edb75c1d6ccf6ce5b49266157c82dd89853f89acc...,8.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000010,0.000000,31.000000,...,0,0,0,0,0,3,3,3.222131,3.2,4
2,0494941a2c1ca07f809b1b964a6260380c6e86821f7182...,8.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000010,0.000000,39.593719,...,0,0,0,0,0,3,2,3.679435,2.8,1
3,13cc2281bc201f3178a9570fcfda9e59a10bdb694a4a94...,6.0,0.000000,0.000000,45.000000,3.833333,0.166667,2297.309398,49.000000,57.328434,...,1,1,0,1,0,3,3,2.489784,3.2,-1
4,4caa6eb48b9278137f7de9468c4a443779b072037916dd...,8.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000010,0.000000,99.849337,...,0,0,0,0,0,3,1,2.892218,2.4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,d3a5339bdc40dbe1d4d1272da82c5460ce3fe7bd9715af...,8.0,0.997928,92.500000,0.166667,0.000000,0.000000,9.023148,0.166667,26.335590,...,0,0,0,0,0,2,2,3.108554,2.8,1
187,f5deb2394f8379cb1a01a52fea6ae320126faccdcd5646...,8.0,1.000000,0.000000,0.500000,0.000000,0.000000,4.978750,0.500000,99.849337,...,0,0,0,0,0,4,2,2.892218,2.8,4
188,39d1955ec15c989db1f9223f4ec4f4ea0a91c99b487bca...,8.0,0.996094,161.833333,3.000000,0.333333,0.000000,274.104630,3.333333,39.593719,...,0,0,0,0,0,3,2,3.679382,2.8,1
189,35e4d92c817f9243d909103e3be0080ea4ff4ea27a5791...,8.0,1.000000,0.000000,0.333333,0.000000,0.166667,21.267083,0.500000,39.593719,...,0,0,0,0,0,3,2,3.679435,2.8,1


In [28]:
import pandas as pd

multiply_by = lambda x: x * 0.5 if x.name != 'cod app' else x
df_enmascarado = df_clusters.apply(multiply_by)

df_enmascarado

,cod app,"¿Cómo valoras tu satisfacción al usar las capacidades del MO TI en tu día a día; las consideras sencillas, te brindan autonomía y te generan valor?",%Disponibilidad,Min Afectación,Baja,Media,Critica,mttr_hr,Cumple ANS,LT,...,No existe cultura de medición,El cambio no tiene la suficiente credibilidad al momento de generar resultados,El modelo de toma de decisiones para este cambio no es claro,Existen barreras de comunicación en la organización que afectan este cambio,Los objetivos que busca este cambio no están alineados con los objetivos de la organización,Ubicacion Infra,Beneficio Cloud,Medición Beneficio Cloud,Nivel Madurez,Cluster
0,eea15b72e72c0034b32fb3154a07a273e451a437dc8209...,4.0,0.497576,83.583333,0.166667,0.000000,0.166667,33.610949,0.333333,32.323277,...,0.0,0.0,0.0,0.0,0.0,1.5,1.0,1.745459,1.4,1.5
1,423b3edb75c1d6ccf6ce5b49266157c82dd89853f89acc...,4.0,0.500000,0.000000,0.000000,0.000000,0.000000,0.000005,0.000000,15.500000,...,0.0,0.0,0.0,0.0,0.0,1.5,1.5,1.611065,1.6,2.0
2,0494941a2c1ca07f809b1b964a6260380c6e86821f7182...,4.0,0.500000,0.000000,0.000000,0.000000,0.000000,0.000005,0.000000,19.796860,...,0.0,0.0,0.0,0.0,0.0,1.5,1.0,1.839717,1.4,0.5
3,13cc2281bc201f3178a9570fcfda9e59a10bdb694a4a94...,3.0,0.000000,0.000000,22.500000,1.916667,0.083333,1148.654699,24.500000,28.664217,...,0.5,0.5,0.0,0.5,0.0,1.5,1.5,1.244892,1.6,-0.5
4,4caa6eb48b9278137f7de9468c4a443779b072037916dd...,4.0,0.500000,0.000000,0.000000,0.000000,0.000000,0.000005,0.000000,49.924669,...,0.0,0.0,0.0,0.0,0.0,1.5,0.5,1.446109,1.2,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,d3a5339bdc40dbe1d4d1272da82c5460ce3fe7bd9715af...,4.0,0.498964,46.250000,0.083333,0.000000,0.000000,4.511574,0.083333,13.167795,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.554277,1.4,0.5
187,f5deb2394f8379cb1a01a52fea6ae320126faccdcd5646...,4.0,0.500000,0.000000,0.250000,0.000000,0.000000,2.489375,0.250000,49.924669,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.446109,1.4,2.0
188,39d1955ec15c989db1f9223f4ec4f4ea0a91c99b487bca...,4.0,0.498047,80.916667,1.500000,0.166667,0.000000,137.052315,1.666667,19.796860,...,0.0,0.0,0.0,0.0,0.0,1.5,1.0,1.839691,1.4,0.5
189,35e4d92c817f9243d909103e3be0080ea4ff4ea27a5791...,4.0,0.500000,0.000000,0.166667,0.000000,0.083333,10.633542,0.250000,19.796860,...,0.0,0.0,0.0,0.0,0.0,1.5,1.0,1.839717,1.4,0.5


In [29]:
df_enmascarado.columns

Index(['cod app',
       '¿Cómo valoras tu satisfacción al usar las capacidades del MO TI en tu día a día; las consideras sencillas, te brindan autonomía y te generan valor?',
       '%Disponibilidad', 'Min Afectación', 'Baja', 'Media', 'Critica',
       'mttr_hr', 'Cumple ANS', 'LT', 'PT', 'Incidentes Onpremise',
       'Incidentes Nube', 'Cambios Onpremise', 'Cambios Nube',
       'Continuidad Onpremise', 'Continuidad Nube', 'Configuración',
       'Habilitar y Mantener Soluciones', 'AMI y Parchado', 'Operaciones TI',
       'Backup & Archiving', 'Optimizar los costos nube',
       'Administrar etiquetas',
       'Administrar estructura de organización y cuentas nube',
       'Nombre del Servicio', 'Nombre de EVC',
       'Falta de tiempo de los líderes',
       'Poco entrenamiento en los temas que implica este cambio',
       'El momento que vive la organización no es pertinente para realizar el cambio',
       'Sentimiento de temor o ansiedad por los cambios que traerá esta transfo

In [30]:
# Define un diccionario de nombres abreviados para las columnas
enmascaramiento_de_columnas = {
    'cod app': 'CA',
    '¿Cómo valoras tu satisfacción al usar las capacidades del MO TI en tu día a día; las consideras sencillas, te brindan autonomía y te generan valor?': 'SVM',
    '%Disponibilidad': 'D',
    'Min Afectación': 'MA',
    'Baja': 'B',
    'Media': 'M',
    'Critica': 'C',
    'mttr_hr': 'MHR',
    'Cumple ANS': 'CANS',
    'LT': 'L',
    'PT': 'P',
    'Incidentes Onpremise': 'IncO',
    'Incidentes Nube': 'IncN',
    'Cambios Onpremise': 'CamO',
    'Cambios Nube': 'CamN',
    'Continuidad Onpremise': 'ConO',
    'Continuidad Nube': 'ConN',
    'Configuración': 'Confi',
    'Habilitar y Mantener Soluciones': 'HyMS',
    'AMI y Parchado': 'AMIP',
    'Operaciones TI': 'OTI',
    'Backup & Archiving': 'BYA',
    'Optimizar los costos nube': 'CostN',
    'Administrar etiquetas': 'AE',
    'Administrar estructura de organización y cuentas nube': 'AEOCN',
    'Nombre del Servicio': 'NServ',
    'Nombre de EVC': 'Nteam',
    'Falta de tiempo de los líderes': 'TieL',
    'Poco entrenamiento en los temas que implica este cambio': 'Ent',
    'El momento que vive la organización no es pertinente para realizar el cambio': 'Mom',
    'Sentimiento de temor o ansiedad por los cambios que traerá esta transformación': 'Tem',
    'No existe cultura de medición': 'CulM',
    'El cambio no tiene la suficiente credibilidad al momento de generar resultados': 'Cre',
    'El modelo de toma de decisiones para este cambio no es claro': 'Mod',
    'Existen barreras de comunicación en la organización que afectan este cambio': 'Bar',
    'Los objetivos que busca este cambio no están alineados con los objetivos de la organización': 'Obj',
    'Ubicacion Infra': 'Inf',
    'Beneficio Cloud': 'BenNube',
    'Medición Beneficio Cloud': 'MedBNube',
    'Nivel Madurez': 'ML'
}


df_enmascarado.rename(columns=enmascaramiento_de_columnas, inplace=True)
df_enmascarado

,CA,SVM,D,MA,B,M,C,MHR,CANS,L,...,CulM,Cre,Mod,Bar,Obj,Inf,BenNube,MedBNube,ML,Cluster
0,eea15b72e72c0034b32fb3154a07a273e451a437dc8209...,4.0,0.497576,83.583333,0.166667,0.000000,0.166667,33.610949,0.333333,32.323277,...,0.0,0.0,0.0,0.0,0.0,1.5,1.0,1.745459,1.4,1.5
1,423b3edb75c1d6ccf6ce5b49266157c82dd89853f89acc...,4.0,0.500000,0.000000,0.000000,0.000000,0.000000,0.000005,0.000000,15.500000,...,0.0,0.0,0.0,0.0,0.0,1.5,1.5,1.611065,1.6,2.0
2,0494941a2c1ca07f809b1b964a6260380c6e86821f7182...,4.0,0.500000,0.000000,0.000000,0.000000,0.000000,0.000005,0.000000,19.796860,...,0.0,0.0,0.0,0.0,0.0,1.5,1.0,1.839717,1.4,0.5
3,13cc2281bc201f3178a9570fcfda9e59a10bdb694a4a94...,3.0,0.000000,0.000000,22.500000,1.916667,0.083333,1148.654699,24.500000,28.664217,...,0.5,0.5,0.0,0.5,0.0,1.5,1.5,1.244892,1.6,-0.5
4,4caa6eb48b9278137f7de9468c4a443779b072037916dd...,4.0,0.500000,0.000000,0.000000,0.000000,0.000000,0.000005,0.000000,49.924669,...,0.0,0.0,0.0,0.0,0.0,1.5,0.5,1.446109,1.2,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,d3a5339bdc40dbe1d4d1272da82c5460ce3fe7bd9715af...,4.0,0.498964,46.250000,0.083333,0.000000,0.000000,4.511574,0.083333,13.167795,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.554277,1.4,0.5
187,f5deb2394f8379cb1a01a52fea6ae320126faccdcd5646...,4.0,0.500000,0.000000,0.250000,0.000000,0.000000,2.489375,0.250000,49.924669,...,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.446109,1.4,2.0
188,39d1955ec15c989db1f9223f4ec4f4ea0a91c99b487bca...,4.0,0.498047,80.916667,1.500000,0.166667,0.000000,137.052315,1.666667,19.796860,...,0.0,0.0,0.0,0.0,0.0,1.5,1.0,1.839691,1.4,0.5
189,35e4d92c817f9243d909103e3be0080ea4ff4ea27a5791...,4.0,0.500000,0.000000,0.166667,0.000000,0.083333,10.633542,0.250000,19.796860,...,0.0,0.0,0.0,0.0,0.0,1.5,1.0,1.839717,1.4,0.5


In [31]:
reemplazo_nombre_cluster= { -0.5: 'Cluster de adaptación',
                           0: 'Cluster Estrategia Optimizada',
                           0.5: 'Cluster Transición en Progreso',
                           1: 'Cluster Eficiencia en Entrega',
                           1.5: 'Cluster Avance y rápida recuperación',
                           2: 'Cluster Desafío en la Entrega'}

In [32]:
df_enmascarado['Nombre Cluster'] = df_enmascarado['Cluster'].map(reemplazo_nombre_cluster)
df_enmascarado

,CA,SVM,D,MA,B,M,C,MHR,CANS,L,...,Cre,Mod,Bar,Obj,Inf,BenNube,MedBNube,ML,Cluster,Nombre Cluster
0,eea15b72e72c0034b32fb3154a07a273e451a437dc8209...,4.0,0.497576,83.583333,0.166667,0.000000,0.166667,33.610949,0.333333,32.323277,...,0.0,0.0,0.0,0.0,1.5,1.0,1.745459,1.4,1.5,Cluster Avance y rápida recuperación
1,423b3edb75c1d6ccf6ce5b49266157c82dd89853f89acc...,4.0,0.500000,0.000000,0.000000,0.000000,0.000000,0.000005,0.000000,15.500000,...,0.0,0.0,0.0,0.0,1.5,1.5,1.611065,1.6,2.0,Cluster Desafío en la Entrega
2,0494941a2c1ca07f809b1b964a6260380c6e86821f7182...,4.0,0.500000,0.000000,0.000000,0.000000,0.000000,0.000005,0.000000,19.796860,...,0.0,0.0,0.0,0.0,1.5,1.0,1.839717,1.4,0.5,Cluster Transición en Progreso
3,13cc2281bc201f3178a9570fcfda9e59a10bdb694a4a94...,3.0,0.000000,0.000000,22.500000,1.916667,0.083333,1148.654699,24.500000,28.664217,...,0.5,0.0,0.5,0.0,1.5,1.5,1.244892,1.6,-0.5,Cluster de adaptación
4,4caa6eb48b9278137f7de9468c4a443779b072037916dd...,4.0,0.500000,0.000000,0.000000,0.000000,0.000000,0.000005,0.000000,49.924669,...,0.0,0.0,0.0,0.0,1.5,0.5,1.446109,1.2,2.0,Cluster Desafío en la Entrega
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,d3a5339bdc40dbe1d4d1272da82c5460ce3fe7bd9715af...,4.0,0.498964,46.250000,0.083333,0.000000,0.000000,4.511574,0.083333,13.167795,...,0.0,0.0,0.0,0.0,1.0,1.0,1.554277,1.4,0.5,Cluster Transición en Progreso
187,f5deb2394f8379cb1a01a52fea6ae320126faccdcd5646...,4.0,0.500000,0.000000,0.250000,0.000000,0.000000,2.489375,0.250000,49.924669,...,0.0,0.0,0.0,0.0,2.0,1.0,1.446109,1.4,2.0,Cluster Desafío en la Entrega
188,39d1955ec15c989db1f9223f4ec4f4ea0a91c99b487bca...,4.0,0.498047,80.916667,1.500000,0.166667,0.000000,137.052315,1.666667,19.796860,...,0.0,0.0,0.0,0.0,1.5,1.0,1.839691,1.4,0.5,Cluster Transición en Progreso
189,35e4d92c817f9243d909103e3be0080ea4ff4ea27a5791...,4.0,0.500000,0.000000,0.166667,0.000000,0.083333,10.633542,0.250000,19.796860,...,0.0,0.0,0.0,0.0,1.5,1.0,1.839717,1.4,0.5,Cluster Transición en Progreso


In [33]:
csv_filename = 'Dataset_enmascarado.csv'
folder_name = 'Modelo_analiticio_Medicion_Madurez'
df_enmascarado.to_csv(csv_filename)

 

s3_key = f'{folder_name}/{csv_filename}'

 

try:
    s3.upload_file(csv_filename, bucket_name, s3_key)
    print(f"{csv_filename} cargado exitosamente en la carpeta {folder_name} en S3.")
except NoCredentialsError:
    print("No se encontraron credenciales de AWS o se han configurado incorrectamente.")

Dataset_enmascarado.csv cargado exitosamente en la carpeta Modelo_analiticio_Medicion_Madurez en S3.
